In [12]:
from bs4 import BeautifulSoup
import requests
from tqdm import tqdm
import pandas as pd 
import re
import os


In [13]:
def isEnglish(s):
    """Takes a token and checks if the input token is english or other language
        Input (str): input token
        output (bool): true if english else false"""
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return bool(re.findall('[A-Za-z]', s))
    else:
        return True


In [14]:
def remove_duplicates(filename):
    data = pd.read_csv(filename, encoding ='utf-16')
    data = data[['date', 'author','title', 'content']]
    data = data.drop_duplicates()
    data.to_csv(filename, header=['date', 'author','title', 'content'], encoding="utf-16")

In [15]:
url_list = list()
url_list.append({"url":"https://ecadforum.com/Amharic/archives/%s/", "max_page":19743}) # 1124

def spiderECADF(max_pages, page_url, cookies=None):
    data = list()
    for i in tqdm(range(3879, max_pages)):
        url = page_url%i
        if cookies != None:
            crawl_url = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, cookies=cookies)
        else:
            crawl_url = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
            
        if crawl_url.status_code == 404:
            continue
        soup = BeautifulSoup(crawl_url.text, 'lxml')
        body = soup.find("div", class_="post-inner")
        if body == None:
            continue
        title = body.h1.text
        title = " ".join(title.split())
        content = " ".join(s for s in body.find("div", class_="entry").text.split() if not isEnglish(s))
        meta_data = body.find("p", class_="post-meta")
        if meta_data != None:
            author = meta_data.find("span", class_="post-meta-author").text
            date = meta_data.find("span", class_="tie-date").text
        else:
            author, date = None, None
        data.append([date, author, title, content])
        df = pd.DataFrame(data, columns = ['date', 'author', 'title', 'content'])
        df = df[['date', 'author','title', 'content']]       
        out_file = 'ECADF_amharic.csv' 
        if i % 100:
            # if file does not exist write header            
            if not os.path.isfile(out_file):
                df.to_csv(out_file, header=['date', 'author','title', 'content'], encoding="utf-16")
            else: # else it exists so append without writing the header
                df.to_csv(out_file, mode='a', header=False, encoding="utf-16")
        if i%500:
            remove_duplicates(out_file)

In [ ]:
for site in url_list:
    spiderECADF(site['max_page'], site['url'])
    
#     df = pd.DataFrame(data, columns =['title', 'content']) 
#     df.to_csv("ESAT_"+site['category']+".csv")

 15%|██████████▋                                                              | 2320/15864 [1:03:17<6:29:17,  1.72s/it]

In [5]:
data = pd.read_csv("ECADF_amharic.csv", encoding ='utf-16')
data = data[['date', 'author','title', 'content']]

In [6]:
data.head()

,date,author,title,content
0,"April 26, 2011",Posted by: ecadforum,“እኔ ያመጣሁት የአቋምም ሆነ የአቋቋም ለውጥ የለም” አቤ ቶኪቻው,“እኔ ያመጣሁት የአቋምም ሆነ የአቋቋም ለውጥ የለም” አቤ ቶኪቻው ቀደም ...
1,"April 26, 2011",Posted by: ecadforum,“እኔ ያመጣሁት የአቋምም ሆነ የአቋቋም ለውጥ የለም” አቤ ቶኪቻው,“እኔ ያመጣሁት የአቋምም ሆነ የአቋቋም ለውጥ የለም” አቤ ቶኪቻው ቀደም ...
2,"May 8, 2011",Posted by: ecadforum,ኢትዮጵያዊው ሞሃመድ ቡኣዚዝ,ፍቅር ይበልጣል የዐረቡን ዓለም በማንቀጥቀጥ ላይ የሚገኘውን ሕዝባዊ አመፅ...
3,"April 26, 2011",Posted by: ecadforum,“እኔ ያመጣሁት የአቋምም ሆነ የአቋቋም ለውጥ የለም” አቤ ቶኪቻው,“እኔ ያመጣሁት የአቋምም ሆነ የአቋቋም ለውጥ የለም” አቤ ቶኪቻው ቀደም ...
4,"May 8, 2011",Posted by: ecadforum,ኢትዮጵያዊው ሞሃመድ ቡኣዚዝ,ፍቅር ይበልጣል የዐረቡን ዓለም በማንቀጥቀጥ ላይ የሚገኘውን ሕዝባዊ አመፅ...


In [7]:
data = data.drop_duplicates()

In [9]:
data.size

2396

In [10]:
data.to_csv('ECADF_amharic.csv', header=['date', 'author','title', 'content'], encoding="utf-16")